# Object Localization & Semantic Segmentation

In addition to classifying images (this is an image of a dog, this is an image of a cat...) CNN's can be used to determine the location of something in an image. Generally speaking localization entails bounding the object in question with a tight rectangle. There are multiple forms of this, including:

* Single object localization: identify a single object and localize it within a bounding box.
* Multiple object localization (sometimes called Object Detection): Identify and bound all relevant objects within a rectangle (i.e one bounding box per object)
* Semantic segmentation: identify exactly which pixels belong to objects (instead of a bounding box)

![](img/pet_annotations.jpg)

> Image source Oxford Pet dataset: http://www.robots.ox.ac.uk/~vgg/data/pets/

For each of these three categories, a network might be designed to detect many different classes of objects or only identify a single kind of object. For example, you could design a network that:

* Only detects the location of a single dog. 
* Detects the location of a dog, or a cat, and classifies it as such.
* Detects the location of all dogs and cats, but doesn't classify them as dog or cat.
* Detects the location of all dogs and cats and labels each instance as dog or cat. 
* ... and so on ...

The same networks that perform well on classification tasks can often be re-purposed to perform localization, but  the output layers need to change significantly. As you might imagine, the complexity of our solutions increase as the complexity of the task increases: Detecting the location of the dog in a dataset that's all images of dogs is much easier than localizing all of the objects beloinging to 1 of 1000 target classes. Similarly, detecting a bounding box is easier than detecting exactly which pixels contain the object in question.

In this section we are going to limit ourselves to **single object localization** on the implementation side. We will discuss changes that allow for multiple object detection/localization as well as segmentation. Additional study materials are provided in the readme regarding multiple object localization.


